<a href="https://colab.research.google.com/github/Milechwan/if1015/blob/master/Projeto2_IF1015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import copy

dataset = pd.read_csv('https://raw.githubusercontent.com/Milechwan/if1015/master/googleplaystore.csv',encoding = "ISO-8859-1")

In [0]:
dataset.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite â FREE Live Cool Themes, Hid...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Objetivo: Descobrir qual o melhor modelo para predizer a nota de avaliação (rating) dos 
aplicativos

In [3]:
dataset[dataset['Rating'].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
23,Mcqueen Coloring pages,ART_AND_DESIGN,NaN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up
113,Wrinkles and rejuvenation,BEAUTY,NaN,182,5.7M,"100,000+",Free,0,Everyone 10+,Beauty,"September 20, 2017",8.0,3.0 and up
123,Manicure - nail design,BEAUTY,NaN,119,3.7M,"50,000+",Free,0,Everyone,Beauty,"July 23, 2018",1.3,4.1 and up
126,Skin Care and Natural Beauty,BEAUTY,NaN,654,7.4M,"100,000+",Free,0,Teen,Beauty,"July 17, 2018",1.15,4.1 and up
129,"Secrets of beauty, youth and health",BEAUTY,NaN,77,2.9M,"10,000+",Free,0,Mature 17+,Beauty,"August 8, 2017",2.0,2.3 and up
130,Recipes and tips for losing weight,BEAUTY,NaN,35,3.1M,"10,000+",Free,0,Everyone 10+,Beauty,"December 11, 2017",2.0,3.0 and up
134,"Lady adviser (beauty, health)",BEAUTY,NaN,30,9.9M,"10,000+",Free,0,Mature 17+,Beauty,"January 24, 2018",3.0,3.0 and up
163,Anonymous caller detection,BOOKS_AND_REFERENCE,NaN,161,2.7M,"10,000+",Free,0,Everyone,Books & Reference,"July 13, 2018",1.0,2.3 and up
180,SH-02J Owner's Manual (Android 8.0),BOOKS_AND_REFERENCE,NaN,2,7.2M,"50,000+",Free,0,Everyone,Books & Reference,"June 15, 2018",3.0,6.0 and up
185,URBANO V 02 instruction manual,BOOKS_AND_REFERENCE,NaN,114,7.3M,"100,000+",Free,0,Everyone,Books & Reference,"August 7, 2015",1.1,5.1 and up


Primeiramente, deve-se remover os dados faltantes e dados duplicados (comparando pelo nome)

In [4]:
dataset = dataset.dropna(subset=['Rating'])
dataset.drop_duplicates(subset=['App'],keep='last', inplace=True)
print(len(dataset))

8197


Transformação de colunas categóricas (definidas no Projeto 1), e remoção do símbolo $ da coluna Price para transformar em tipo numérico. Adicionalmente, é removida a linha (conforme foi também visto no Projeto 1) que possui os valores nas colunas incorretas, para transformar a coluna Reviews em tipo numérico.

In [0]:
dataset = dataset.drop(dataset[dataset['Price']=='Everyone'].index, axis = 0)
dataset['Reviews'] = pd.to_numeric(dataset['Reviews'])

In [0]:
dataset['Price'] = dataset['Price'].apply(lambda x: x[x.find('$')+1:] if x.find('$')>-1 else x)
dataset['Price'] = pd.to_numeric(dataset['Price'])
#dataset['Last Updated'] = pd.to_datetime(dataset['Last Updated'])

In [0]:
dataset['Type'] = dataset['Type'].astype("category")
dataset['Content Rating'] = dataset['Content Rating'].astype("category") 
dataset['Category'] = dataset['Category'].astype("category") 
dataset['Installs'] = dataset['Installs'].astype("category")
dataset['Android Ver'] = dataset['Android Ver'].astype("category")
dataset['Current Ver'] = dataset['Current Ver'].astype("category")
dataset['Size'] = dataset['Size'].astype("category")

dataset['Type'] = dataset['Type'].cat.codes
dataset['Content Rating'] = dataset['Content Rating'].cat.codes
dataset['Category'] = dataset['Category'].cat.codes
dataset['Installs'] = dataset['Installs'].cat.codes
dataset['Android Ver'] = dataset['Android Ver'].cat.codes
dataset['Current Ver'] = dataset['Current Ver'].cat.codes
dataset['Size'] = dataset['Size'].cat.codes

Remoção de outliers com DBSCAN

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

#Normalizando dataset para que o DBSCAN funcione corretamente para calcular as distâncias e clusterizar os dados
#removendo colunas inutilizadas 
dataset = dataset.drop('Genres', axis = 1)
dataset = dataset.drop('App', axis = 1)
dataset = dataset.drop('Last Updated', axis = 1)

normalized_ds = copy.deepcopy(dataset)
normalized_ds = scaler.fit_transform(normalized_ds)
normalized_ds = pd.DataFrame(normalized_ds,columns=["Category","Rating","Reviews","Size","Installs","Type","Price","Content Rating","Current Ver","Android Ver"])
normalized_ds.head()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Current Ver,Android Ver
0,0.0,0.775,0.000002,0.116223,0.277778,0.0,0.0,0.2,0.041143,0.483871
1,0.0,0.925,0.001120,0.808717,0.666667,0.0,0.0,0.2,0.161143,0.483871
2,0.0,0.875,0.002760,0.213075,0.833333,0.0,0.0,0.8,0.979048,0.580645
3,0.0,0.825,0.000012,0.138015,0.444444,0.0,0.0,0.2,0.095619,0.645161
4,0.0,0.850,0.000002,0.489104,0.777778,0.0,0.0,0.2,0.039619,0.258065


In [0]:
from sklearn.cluster import DBSCAN
outlier_detection = DBSCAN(
  eps = 0.3,
  metric="euclidean",
  min_samples = 10)
clusters = outlier_detection.fit_predict(normalized_ds)

dataset_without_outliers = copy.deepcopy(dataset) #criando cópia para comparar resultado dos modelos com e sem os outliers 

In [11]:
#labels guardam os valores que identificam se determinado item do dataset é ou não um outlier.
labels = outlier_detection.labels_

#verificação do número de grupos (clusters) identificados pelo DBSCAN 
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print("Total clusters: "+str(n_clusters_))
n_noise_ = list(labels).count(-1)#outliers 
print("Total outliers: "+str(n_noise_))

Total clusters: 5
Total outliers: 514


In [12]:
#identificando os índices dos itens que são outliers para removê-los do conjunto de dados
index_outlier = np.where(labels == -1)
result = index_outlier[0]
print(result.shape)

(514,)


In [13]:
dataset_without_outliers.drop(dataset_without_outliers.index[result], inplace = True)
print(len(dataset))
print(len(dataset_without_outliers))

8196
7682



Modelo de predição Linear Regression (sem remover outliers)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

model = LinearRegression()

dataset_without_rating = copy.deepcopy(dataset)
dataset_without_rating = dataset_without_rating.drop('Rating',axis=1)
X = np.array(dataset_without_rating)
y = np.array(dataset['Rating'])
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33,
                                                random_state=42)

In [15]:
model.fit(Xtrain,ytrain)
y_model = model.predict(Xtest)
y_pred_train = model.predict(Xtrain)
print(model.score(Xtest,ytest))

0.011415686779958745


In [18]:
from sklearn.metrics import explained_variance_score, max_error, mean_absolute_error, mean_squared_log_error

print('Explained variance score: '+ str(explained_variance_score(ytest,y_model)))
print('Max error: '+ str(max_error(ytest,y_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,y_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,y_model)))

Explained variance score: 0.01203767049009108
Max error: 3.2094486610562987
Mean absolute error: 0.3724319077580764
Mean squared log error: 0.012924232167634985


Linear Regression (com remoção de outliers)

In [19]:
without_outliers_copy = copy.deepcopy(dataset_without_outliers)
X_without_outliers = np.array(without_outliers_copy.drop('Rating',axis=1))
y_without_outliers = np.array(without_outliers_copy['Rating'])
X_without_out_train, X_without_out_test, y_without_out_train, y_without_out_test = train_test_split(X, y, test_size=0.33,
                                                random_state=42)

model_without_outliers = LinearRegression()
model_without_outliers.fit(X_without_out_train,y_without_out_train)
y_model_without_outliers = model_without_outliers.predict(X_without_out_test)
print(model_without_outliers.score(X_without_out_test,y_without_out_test))

0.011415686779958745


In [20]:
print('Explained variance score: '+ str(explained_variance_score(y_without_out_test,y_model_without_outliers)))
print('Max error: '+ str(max_error(y_without_out_test,y_model_without_outliers)))
print('Mean absolute error: '+str(mean_absolute_error(y_without_out_test,y_model_without_outliers)))
print('Mean squared log error: '+str(mean_squared_log_error(y_without_out_test,y_model_without_outliers)))

Explained variance score: 0.01203767049009108
Max error: 3.2094486610562987
Mean absolute error: 0.3724319077580764
Mean squared log error: 0.012924232167634985


Usando SelectFromModel, que dado o estimador e as variáveis X e y, ele retorna um array contendo toda a base mas selecionando as variáveis independentes mais adequadas para fazer a predição. Método de redução de dimensionalidade

In [0]:
from sklearn.feature_selection import SelectFromModel
feature = SelectFromModel(model)#sem remover outliers
Fit = feature.fit_transform(X, y)

In [22]:
print(Fit.shape)

(8196, 1)


In [23]:
Fittrain, Fittest, yfittrain, yfittest = train_test_split(Fit, y, test_size=0.33,
                                                random_state=42)
model.fit(Fittrain,yfittrain)
y_model_fit = model.predict(Fittest)
print(model.score(Fittest,yfittest))

0.002142380562441204


In [26]:
print('Explained variance score: '+ str(explained_variance_score(yfittest,y_model_fit)))
print('Max error: '+ str(max_error(yfittest,y_model_fit)))
print('Mean absolute error: '+str(mean_absolute_error(yfittest,y_model_fit)))
print('Mean squared log error: '+str(mean_squared_log_error(yfittest,y_model_fit)))

Explained variance score: 0.0027433949763074894
Max error: 3.1623034591194967
Mean absolute error: 0.37618109848603193
Mean squared log error: 0.013036935206045213


RandomForest

In [0]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

Random Forest sem remoção de outliers

In [34]:
rf.fit(Xtrain,ytrain)

y_rf_model = rf.predict(Xtest)

print('R² score: '+ str(rf.score(Xtest,ytest)))
print('Explained variance score: '+ str(explained_variance_score(ytest,y_rf_model)))
print('Max error: '+ str(max_error(ytest,y_rf_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,y_rf_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,y_rf_model)))

R² score: 0.0887503657237766
Explained variance score: 0.09352776433575338
Max error: 3.189
Mean absolute error: 0.35653900184842885
Mean squared log error: 0.011873835395163807


Random Forest com remoção de outliers

In [30]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

rf.fit(X_without_out_train,y_without_out_train)
result_rf_without_outliers = rf.predict(X_without_out_test)
print('R² score: '+str(rf.score(X_without_out_test,y_without_out_test)))

print('Explained variance score: '+ str(explained_variance_score(y_without_out_test,result_rf_without_outliers)))
print('Max error: '+ str(max_error(y_without_out_test,result_rf_without_outliers)))
print('Mean absolute error: '+str(mean_absolute_error(y_without_out_test,result_rf_without_outliers)))
print('Mean squared log error: '+str(mean_squared_log_error(y_without_out_test,result_rf_without_outliers)))

R² score: 0.0887503657237766
Explained variance score: 0.09352776433575338
Max error: 3.189
Mean absolute error: 0.35653900184842885
Mean squared log error: 0.011873835395163807


Support Vector Regression - outro modelo de predição apenas para comparar desempenho e ver qual modelo de fato vale a pena tentar melhorar 

In [35]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

svr_rbf.fit(Xtrain,ytrain)
result_svr_rbf = svr_rbf.predict(Xtest)

print('R² score: '+ str(rf.score(Xtest,ytest)))
print('Explained variance score: '+ str(explained_variance_score(ytest,result_svr_rbf)))
print('Max error: '+ str(max_error(ytest,result_svr_rbf)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,result_svr_rbf)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,result_svr_rbf)))

R² score: 0.0887503657237766
Explained variance score: 0.0017416932320917233
Max error: 3.1437504953858086
Mean absolute error: 0.3832565545553754
Mean squared log error: 0.0130425749650298


In [36]:
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

svr_rbf.fit(X_without_out_train,y_without_out_train)
result_svr_rbf = svr_rbf.predict(X_without_out_test)
print('R² score: '+str(svr_rbf.score(X_without_out_test,y_without_out_test)))
print('Explained variance score: '+ str(explained_variance_score(y_without_out_test,result_svr_rbf)))
print('Max error: '+ str(max_error(y_without_out_test,result_svr_rbf)))
print('Mean absolute error: '+str(mean_absolute_error(y_without_out_test,result_svr_rbf)))
print('Mean squared log error: '+str(mean_squared_log_error(y_without_out_test,result_svr_rbf)))

R² score: -0.0035166980482019827
Explained variance score: 0.0017416932320917233
Max error: 3.1437504953858086
Mean absolute error: 0.3832565545553754
Mean squared log error: 0.0130425749650298


Devido aos resultados terem valores relativamente próximos entre si, mostrando que na verdade os modelos estão "enviesados" (tendo score muito próimo de 0), o modelo que apresentou-se ligeiramente melhor foi RandomForest, já que seu score foi o maior de todos, ainda que muito próximo de 0. Vimos também que remover outliers não influencia no desempenho dos modelos. Daqui em diante, será usado GridSearch para tentar melhorar esse modelo de predição 

In [40]:
from sklearn.metrics import confusion_matrix

confusion_matrix(ytest, result_svr_rbf)

ValueError: ignored